# Setup environment

In [1]:
# Essentials
import os
import numpy as np
import importlib
import json
from datetime import datetime

os.environ["GIT_PYTHON_GIT_EXECUTABLE"] = "/usr/bin/git"

import git
git.refresh("/usr/bin/git")

from foolbox import PyTorchModel

# PyTorch
import torch
from torch.utils.data import DataLoader

# Utils
import utils
importlib.reload(utils)

from utils import get_files, save_image, make_dirs, get_model, select_gpu, get_data, get_class_weigths
from utils import CustomTransforms, My_data, FocalLoss

# OnePixelAttack
import OnePixelAttack
importlib.reload(OnePixelAttack)

# TriangleAttack
import TriangleAttack
importlib.reload(TriangleAttack)
from TriangleAttack import TA

import ProjectedGradientDescent
importlib.reload(ProjectedGradientDescent)

<module 'ProjectedGradientDescent' from '/home/jovyan/ProjectedGradientDescent.py'>

## Setup Cuda

In [2]:
# Set a higher max split size to avoid memory problems
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [3]:
device = torch.device(f"cuda:{select_gpu()}" if torch.cuda.is_available() else "cpu")

print(device)

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(torch.cuda.memory_summary(device=None, abbreviated=False))
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False

Selecting GPU 0 with 22729MB free memory
cuda:0
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from lar

## Setup Transformers

In [4]:
custom_transforms = CustomTransforms()
resize_transform = custom_transforms.get_transform("resize_tensor")
test_transform = custom_transforms.get_transform("test")

## Get the data

In [5]:
# Load train and test files that are used for the model.
train_dict, test_dict = get_data(os.path.join(os.getcwd(), "BreaKHis_v1"))

Opening /home/jovyan/txt/train.txt.txt
Opening /home/jovyan/txt/test.txt.txt


In [6]:
test_files = "./dataset/test/original/**/**/*.png"

In [7]:
org_norm = My_data(get_files(test_files), transforms=test_transform)
org_dataloader = DataLoader(org_norm)

## Setup network

In [8]:
model_name = "swin"
model = get_model(device, model_name)
model.eval()

/home/jovyan/.local/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


SwinTransformerV2(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerV2Stage(
      (downsample): Identity()
      (blocks): ModuleList(
        (0): SwinTransformerV2Block(
          (attn): WindowAttention(
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=3, bias=False)
            )
            (qkv): Linear(in_features=96, out_features=288, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (drop_path1): Identi

## Setup performance metrics

In [9]:
filepaths = {
    "Network": ["SWIN", "RESNET", "RETRAINED SWIN", "RETRAINED RESNET"],
    "Attack": ["Regular", "OnePixel", "Triangle", "PGD"]
}
network = 2

# Evaluate performance

## Evaluate model performance

In [ ]:
plot_data = {
    str(i): [] for i in range(org_norm.__len__())
}

In [ ]:
with torch.no_grad():
    for i, (image, label) in enumerate(org_dataloader):
        path = org_norm.__getpath__(i)
        true_label = torch.argmax(label.to(device), dim=1)
        confs = model(image.to(device))
        pred_label = torch.argmax(confs, dim=1)
    
        plot_data[str(i)] = [path, true_label.item(), pred_label.item(), confs.cpu().numpy()[0].tolist()]

In [ ]:
name = "{} - {} - {}".format(filepaths["Network"][network], filepaths["Attack"][0], datetime.now().strftime("%Y-%m-%d %H%M"))

with open("{}.txt".format(name), "w") as output:
    output.write(json.dumps(plot_data))

## Perform and evaluate Advanced OnePixel Attack

In [ ]:
with torch.no_grad():
        op_advs = OnePixelAttack.attack(1, model, device, org_dataloader, pixel_count=1, maxiter=50, popsize=15)

In [ ]:
plot_data = {
    str(i): [] for i in range(org_norm.__len__())
}

In [ ]:
make_dirs("test/one_pixel_attack/"+model_name)
with torch.no_grad():
    for i, (image, label) in enumerate(org_dataloader):
        path = org_norm.__getpath__(i)
        true_label = torch.argmax(label.to(device), dim=1)
        confs = model(op_advs[i].to(device))
        pred_label = torch.argmax(confs, dim=1)
        save_image(op_advs[i][0], "./dataset/test/one_pixel_attack/"+model_name+"/" + path)
        plot_data[str(i)] = [path, true_label.item(), pred_label.item(), confs.cpu().numpy()[0].tolist()]

In [ ]:
name = "{} - {} - {}".format(filepaths["Network"][network], filepaths["Attack"][1], datetime.now().strftime("%Y-%m-%d %H%M"))

with open("{}.txt".format(name), "w") as output:
    output.write(json.dumps(plot_data))

## Perform and evaluate Triangle attack   

In [10]:
#model = PyTorchModel(get_model(device, model_name).eval(), bounds=(0,1), device=device)
pt_model = PyTorchModel(model.eval(), bounds=(0,1), device=device)
# run on a10 gpu as it has highest memory, is processes about 100 images in 20 mins for the SWIN
print("Attack !")

with torch.no_grad():
    ta_model = TA(pt_model, input_device=device)
    my_advs, q_list, my_intermediates, max_length = ta_model.attack(org_dataloader)
    print('TA Attack Done')

Attack !
torch.Size([777])
calculating best_advs


/home/jovyan/.local/lib/python3.8/site-packages/foolbox/attacks/blended_noise.py:85: UserWarning: LinearSearchBlendedUniformNoiseAttack failed to draw sufficient random inputs that are adversarial (677 / 777).
  warnings.warn(



 tensor([[[0.6579, 0.5589, 0.6566,  ..., 0.7239, 0.6877, 0.5680],
         [0.5562, 0.6248, 0.7855,  ..., 0.7351, 0.5897, 0.5855],
         [0.6280, 0.6387, 0.6573,  ..., 0.6125, 0.7321, 0.5811],
         ...,
         [0.6108, 0.7750, 0.7223,  ..., 0.8286, 0.6866, 0.6795],
         [0.6518, 0.6380, 0.8116,  ..., 0.8481, 0.6523, 0.7771],
         [0.6669, 0.6578, 0.8325,  ..., 0.6708, 0.7974, 0.7978]],

        [[0.6540, 0.6751, 0.5069,  ..., 0.6829, 0.7618, 0.6793],
         [0.4954, 0.5819, 0.5188,  ..., 0.5853, 0.5694, 0.5888],
         [0.5417, 0.5391, 0.6453,  ..., 0.5305, 0.7139, 0.6500],
         ...,
         [0.6862, 0.8243, 0.8488,  ..., 0.8874, 0.8536, 0.8899],
         [0.6738, 0.7781, 0.7831,  ..., 0.8612, 0.8644, 0.7013],
         [0.7440, 0.7511, 0.6894,  ..., 0.8423, 0.8176, 0.7639]],

        [[0.5422, 0.5797, 0.5631,  ..., 0.6488, 0.7633, 0.6577],
         [0.6668, 0.7013, 0.5851,  ..., 0.7582, 0.7463, 0.6191],
         [0.6968, 0.6339, 0.6224,  ..., 0.7138, 0.7137, 

In [12]:
plot_data = {
    str(i): [] for i in range(len(my_advs))
}

make_dirs("test/triangle_attack/"+model_name)
for i in range(len(my_advs)):
    path = org_norm.__getpath__(i)
    true_label = torch.argmax(org_norm.__getitem__(i)[1]).item()
    confs = model(my_advs[i].unsqueeze(0))
    pred_label = torch.argmax(confs, dim=1)
    
    save_image(my_advs[i], "./dataset/test/triangle_attack/"+model_name+"/" + path)
    plot_data[str(i)] = [path, true_label, pred_label.item(), confs.detach().cpu().numpy()[0].tolist()]

In [13]:
name = "{} - {} - {}".format(filepaths["Network"][network], filepaths["Attack"][2], datetime.now().strftime("%Y-%m-%d %H%M"))

with open("{}.txt".format(name), "w") as output:
    output.write(json.dumps(plot_data))

## Perform and evaluate Project Gradient Descent attack

In [14]:
plot_data = {
    str(i): [] for i in range(org_norm.__len__())
}
# Avoid using with torch.no_grad()
pgd_advs = ProjectedGradientDescent.pgd_attack(org_dataloader, model, device, get_class_weigths(train_dict).to(device))

In [15]:
make_dirs("test/pgd_attack/"+model_name)
with torch.no_grad():
    for i, (image, label) in enumerate(org_dataloader):
        path = org_norm.__getpath__(i)
        true_label = torch.argmax(label.to(device), dim=1)
        confs = model(pgd_advs[i].to(device))
        pred_label = torch.argmax(confs, dim=1)
        save_image(pgd_advs[i][0], "./dataset/test/pgd_attack/"+model_name+"/" + path)
        plot_data[str(i)] = [path, true_label.item(), pred_label.item(), confs.cpu().numpy()[0].tolist()]

In [16]:
name = "{} - {} - {}".format(filepaths["Network"][network], filepaths["Attack"][3], datetime.now().strftime("%Y-%m-%d %H%M"))

with open("{}.txt".format(name), "w") as output:
    output.write(json.dumps(plot_data))